In [21]:
import pandas as pd
import json
import myql
from yahoo_oauth import OAuth1

In [22]:
cols = ['Name', 'Team', 'GP', 'G', 'A', 'Pts', 'PIM', 'PPP', 'SOG', 'FW', 'Hits']
# reads 2012 data
df12 = pd.read_csv('Stats/Fantasy Hockey Stats - 2012.csv')
df12['PPP'] = df12.PPG+df12.PPA
df12 = df12[cols]
# reads 2013 data
df13 = pd.read_csv('Stats/Fantasy Hockey Stats - 2013.csv')
df13['PPP'] = df13.PPG+df13.PPA
df13 = df13[cols]
# reads 2014 data
df14 = pd.read_csv('Stats/Fantasy Hockey Stats - 2014.csv')
df14['PPP'] = df14.PPG+df14.PPA
df14 = df14[cols]
# reads 2015 data
df15 = pd.read_html(
    "http://sports.yahoo.com/nhl/stats/byposition?pos=C%2CRW%2CLW%2CD&sort=14&conference=NHL&year=season_2015",
    match = 'Name', flavor = 'bs4', header = 0)[0]
df15['PPP'] = df15.PPG+df15.PPA
df15 = df15[cols]

In [23]:
# reads goalie data
g12 = pd.read_csv('Stats/Fantasy Hockey Stats - g12.csv')
g13 = pd.read_csv('Stats/Fantasy Hockey Stats - g13.csv')
g14 = pd.read_csv('Stats/Fantasy Hockey Stats - g14.csv')
g15 = pd.read_html('http://sports.yahoo.com/nhl/stats/byposition?pos=G&sort=102&conference=NHL&year=season_2015',
    match = 'Name', flavor = 'bs4', header = 0)[0]

In [24]:
# merges data
df = df15.append(df12)
df = df.append(df13)
df = df.append(df14)
df = df.groupby('Name').sum()
gf = g15.append(g12)
gf = gf.append(g13)
gf = gf.append(g14)
gf = gf.groupby('Name').sum()

In [25]:
# creates skater average stats
df['G_avg'] = df['G']/df['GP']
df['A_avg'] = df['A']/df['GP']
df['P_avg'] = df['Pts']/df['GP']
df['PIM_avg'] = df['PIM']/df['GP']
df['PPP_avg'] = df['PPP']/df['GP']
df['SOG_avg'] = df['SOG']/df['GP']
df['FW_avg'] = df['FW']/df['GP']
df['HITS_avg'] = df['Hits']/df['GP']
avg_cols = ['GP', 'G_avg', 'A_avg', 'P_avg', 'PIM_avg', 'PPP_avg', 
            'SOG_avg', 'FW_avg', 'HITS_avg']
skaters = df[avg_cols]

In [26]:
# creates goalie average stats
gf_cols = ['GP', 'MIN', 'SV', 'SA', 'GA']
gf = gf[gf_cols]
gf['SV%'] = gf['SV']/gf['SA']
gf['GAA'] = gf['GA']/ (gf['MIN'] / 60)
gf['POS'] = 'G'
gf['SV_avg'] = gf['SV']/gf['GP']
gfGP = gf['GP']
gf = gf[['GAA', 'SV_avg', 'SV%']]
gf['GAA'] = gf['GAA'] * -1

In [27]:
# adds positions
file = pd.read_csv('Stats/Fantasy Hockey Stats - Defense.csv')
defense = df[avg_cols][df.index.isin(file['Name'])]
forwards = df[avg_cols][~df.index.isin(file['Name'])]

# limits to players active this season
defense = defense[defense.index.isin(df15['Name'])]
forwards = forwards[forwards.index.isin(df15['Name'])]
goalies = gf[gf.index.isin(g15['Name'])]

In [28]:
# strips and saves games played data
skatersGP = skaters.GP
skaters = skaters[skaters.columns[1:]]
defenseGP = defense.GP
defense = defense[defense.columns[1:]]
forwardsGP = forwards.GP
forwards = forwards[forwards.columns[1:]]

In [29]:
# normalizes data

defense_norm = (defense - defense.mean()) / defense.std()
defense_norm['Score'] = defense_norm.sum(axis=1)
defense_norm['GP'] = defenseGP

forwards_norm = (forwards - forwards.mean()) / forwards.std()
forwards_norm['Score'] = forwards_norm.sum(axis=1)
forwards_norm['GP'] = forwardsGP

goalies_norm = (goalies - goalies.mean()) / goalies.std()
goalies_norm['Score'] = goalies_norm.sum(axis=1)
goalies_norm['GP'] = gfGP

In [30]:
# ranks by position

defense = defense_norm.sort_values('Score', ascending=False)
rank = list(range(1,len(defense)+1))
defense['PRank'] = rank
defense['POS'] = 'D'
defense = defense[['PRank', 'Score', 'POS', 'GP','G_avg', 'A_avg', 'P_avg', 
                   'PIM_avg', 'PPP_avg', 'SOG_avg', 'FW_avg', 'HITS_avg']]

forwards = forwards_norm.sort_values('Score', ascending=False)
rank = list(range(1,len(forwards)+1))
forwards['PRank'] = rank
forwards['POS'] = 'F'
forwards = forwards[['PRank', 'Score', 'POS', 'GP','G_avg', 'A_avg', 'P_avg',
                     'PIM_avg', 'PPP_avg', 'SOG_avg', 'FW_avg', 'HITS_avg']]

goalies = goalies_norm.sort_values('Score', ascending=False)
rank = list(range(1,len(goalies)+1))
goalies['PRank'] = rank
goalies['POS'] = 'G'
goalies = goalies[['PRank', 'Score', 'POS', 'GP','GAA', 'SV_avg', 'SV%']]

In [31]:
# merges positions and adds overall rank
allskaters = defense.append(forwards)
allskaters = allskaters.append(goalies)
allskaters = allskaters.sort_values('Score', ascending = False)
ORank = list(range(1, len(allskaters)+1))
allskaters['ORank'] = ORank
allskaters['Team'] = 'FA'
allskaters = allskaters[['ORank', 'PRank', 'Score', 'Team', 'POS', 'GP','G_avg', 'A_avg', 
                         'P_avg', 'PIM_avg', 'PPP_avg', 'SOG_avg', 'FW_avg', 'HITS_avg',
                        'GAA', 'SV_avg', 'SV%']]

In [32]:
oauth = OAuth1(None, None, from_file='credentials.json')
yql = myql.MYQL(oauth = oauth)
response = yql.raw_query("select * from fantasysports.teams.roster where league_key='352.l.63158'")
data = response.json()

with open('rosters.json', 'w') as outfile:
    json.dump(data, outfile)

[2016-01-07 10:53:27,711 DEBUG] [yahoo_oauth.yahoo_oauth.__init__] Checking 
[2016-01-07 10:53:27,714 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] ELAPSED TIME : 198.7709629535675
[2016-01-07 10:53:27,715 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] TOKEN IS STILL VALID
[2016-01-07 10:53:27,717 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] ELAPSED TIME : 198.77319598197937
[2016-01-07 10:53:27,717 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] TOKEN IS STILL VALID


In [33]:
# creates dict of team rosters
with open('rosters.json') as data_file:    
    data = json.load(data_file)
teams = []
for num in range(0,14):
    teams.append(data['query']['results']['team'][num]['name'])
count = 0
rosters = {}
for team in teams:
    rosters[team] = []
    for num in range(0, int(data['query']['results']['team'][count]
                            ['roster']['players']['count'])):
        rosters[team].append(data['query']['results']['team'][count]
                             ['roster']['players']['player'][num]['name']['full'])
    count +=1

In [34]:
# creates dataframe of each team and adds team to allskaters
Game_of_Doans = allskaters[allskaters.index.isin(rosters['Game of Doans'])]
Horsedick = allskaters[allskaters.index.isin(rosters['Horsedick.mpeg'])]
Fists_of_Fleury = allskaters[allskaters.index.isin(rosters['Fists of Fleury'])]
Munson_Munson = allskaters[allskaters.index.isin(rosters['Munson Munson'])]
The_Ants = allskaters[allskaters.index.isin(rosters['The Ants'])]
Ricks_Red_Wingels = allskaters[allskaters.index.isin(rosters["Rick's Red Wingels"])]
CamelToews = allskaters[allskaters.index.isin(rosters['CamelToews'])]
Sarahs_Super_Team = allskaters[allskaters.index.isin(rosters["Sarah's Super Team"])]
Nordic_Supermen = allskaters[allskaters.index.isin(rosters['Nordic Supermen'])]
DeKeyser = allskaters[allskaters.index.isin(rosters['DeKeyser Söze'])]
Shiroi_Kabocha = allskaters[allskaters.index.isin(rosters['Shiroi Kabocha'])]
Brooklyn_Solar_Bears = allskaters[allskaters.index.isin(rosters['Brooklyn Solar Bears'])]
STEPANWOLF = allskaters[allskaters.index.isin(rosters['STEPANWOLF'])]
Caress_of_Krieger = allskaters[allskaters.index.isin(rosters['Caress of Krieger'])]

allskaters['Team'][allskaters.index.isin(Game_of_Doans.index)] = 'Game of Doans'
allskaters['Team'][allskaters.index.isin(Horsedick.index)] = 'Horsedick.mpeg'
allskaters['Team'][allskaters.index.isin(Fists_of_Fleury.index)] = 'Fists of Fleury'
allskaters['Team'][allskaters.index.isin(Munson_Munson.index)] = 'Munson Munson'
allskaters['Team'][allskaters.index.isin(The_Ants.index)] = 'The Ants'
allskaters['Team'][allskaters.index.isin(Ricks_Red_Wingels.index)] = "Rick's Red Wingels"
allskaters['Team'][allskaters.index.isin(CamelToews.index)] = 'CamelToews'
allskaters['Team'][allskaters.index.isin(Sarahs_Super_Team.index)] = "Sarah's Super Team"
allskaters['Team'][allskaters.index.isin(Nordic_Supermen.index)] = 'Nordic Supermen'
allskaters['Team'][allskaters.index.isin(DeKeyser.index)] = 'DeKeyser Söze'
allskaters['Team'][allskaters.index.isin(Shiroi_Kabocha.index)] = 'Shiroi Kabocha'
allskaters['Team'][allskaters.index.isin(Brooklyn_Solar_Bears.index)] = 'Brooklyn Solar Bears'
allskaters['Team'][allskaters.index.isin(STEPANWOLF.index)] = 'STEPANWOLF'
allskaters['Team'][allskaters.index.isin(Caress_of_Krieger.index)] = 'Caress of Krieger'

Game_of_Doans = allskaters[allskaters.index.isin(rosters['Game of Doans'])]
Horsedick = allskaters[allskaters.index.isin(rosters['Horsedick.mpeg'])]
Fists_of_Fleury = allskaters[allskaters.index.isin(rosters['Fists of Fleury'])]
Munson_Munson = allskaters[allskaters.index.isin(rosters['Munson Munson'])]
The_Ants = allskaters[allskaters.index.isin(rosters['The Ants'])]
Ricks_Red_Wingels = allskaters[allskaters.index.isin(rosters["Rick's Red Wingels"])]
CamelToews = allskaters[allskaters.index.isin(rosters['CamelToews'])]
Sarahs_Super_Team = allskaters[allskaters.index.isin(rosters["Sarah's Super Team"])]
Nordic_Supermen = allskaters[allskaters.index.isin(rosters['Nordic Supermen'])]
DeKeyser = allskaters[allskaters.index.isin(rosters['DeKeyser Söze'])]
Shiroi_Kabocha = allskaters[allskaters.index.isin(rosters['Shiroi Kabocha'])]
Brooklyn_Solar_Bears = allskaters[allskaters.index.isin(rosters['Brooklyn Solar Bears'])]
STEPANWOLF = allskaters[allskaters.index.isin(rosters['STEPANWOLF'])]
Caress_of_Krieger = allskaters[allskaters.index.isin(rosters['Caress of Krieger'])]

//anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
//anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
//anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
//anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See th

In [35]:
def lookup(*args):
    players = list(args)
    return(allskaters[allskaters.index.isin(players)])

def value(player):
    return lookup(player).append(Game_of_Doans[Game_of_Doans['Score'] 
                                               < list(lookup(player)['Score'])])
def compare(team):
    compare = allskaters.groupby('Team').sum()
    compare = compare.drop('FA')
    compare = compare[['G_avg', 'A_avg', 'P_avg', 'PIM_avg', 'PPP_avg', 
         'SOG_avg', 'FW_avg', 'HITS_avg', 'GAA', 'SV_avg', 'SV%']]
    compare = compare[compare.index.isin([team])].append(
        compare[compare.index.isin(['Game of Doans'])])
    return(compare.diff()[1:])

def rankings():
    df = allskaters.groupby('Team').sum().drop('FA').ix[:,'G_avg':]
    for col in df:
        df[col] = df[col].rank(ascending = False)
    df['Rank'] = df.mean(axis=1)
    df['Rank'] = df['Rank'].rank()
    return(df[['Rank', 'G_avg', 'A_avg', 'P_avg', 'PIM_avg', 'PPP_avg', 
         'SOG_avg', 'FW_avg', 'HITS_avg', 'GAA', 'SV_avg', 'SV%']].sort_values('Rank'))

In [36]:
Game_of_Doans

,ORank,PRank,Score,Team,POS,GP,G_avg,A_avg,P_avg,PIM_avg,PPP_avg,SOG_avg,FW_avg,HITS_avg,GAA,SV_avg,SV%
Name,,,,,,,,,,,,,,,,,
Brent Burns,1,1,24.769485,Game of Doans,D,37,5.393228,1.883961,3.250035,0.402155,3.769078,4.669587,4.996365,0.405078,NaN,NaN,NaN
Shayne Gostisbehere,2,2,17.971185,Game of Doans,D,22,3.741175,1.386280,2.314754,-0.641629,2.844424,1.137662,8.516529,-1.328011,NaN,NaN,NaN
Vladimir Tarasenko,24,12,10.548812,Game of Doans,F,41,3.179742,1.379537,2.440558,0.037894,2.649568,2.619421,-0.672533,-1.085376,NaN,NaN,NaN
Steven Stamkos,32,17,9.236726,Game of Doans,F,40,2.013299,0.617217,1.380137,-0.207622,2.734035,1.552473,1.367221,-0.220034,NaN,NaN,NaN
Shane Doan,39,24,8.423908,Game of Doans,F,30,2.558821,-0.071222,1.202678,1.589906,1.733572,1.199774,-0.512552,0.722931,NaN,NaN,NaN
Rasmus Ristolainen,47,17,8.074020,Game of Doans,D,40,1.670780,1.810099,1.937645,-1.182309,1.989120,1.930185,-0.166542,0.085044,NaN,NaN,NaN
Dion Phaneuf,56,20,7.768146,Game of Doans,D,38,0.281868,1.632418,1.334036,1.123856,1.113039,1.227172,-0.166542,1.222299,NaN,NaN,NaN
Brandon Dubinsky,59,39,7.713048,Game of Doans,F,34,0.205587,1.095074,0.806007,0.765158,0.357370,0.163606,2.404742,1.915504,NaN,NaN,NaN
Tyler Bozak,64,44,7.630555,Game of Doans,F,34,0.847378,1.743017,1.536718,-0.292211,1.443845,0.163606,2.734114,-0.545911,NaN,NaN,NaN


In [37]:
lookup('Mark Streit', 'Rasmus Ristolainen')

,ORank,PRank,Score,Team,POS,GP,G_avg,A_avg,P_avg,PIM_avg,PPP_avg,SOG_avg,FW_avg,HITS_avg,GAA,SV_avg,SV%
Name,,,,,,,,,,,,,,,,,
Rasmus Ristolainen,47,17,8.074020,Game of Doans,D,40,1.670780,1.810099,1.937645,-1.182309,1.989120,1.930185,-0.166542,0.085044,NaN,NaN,NaN
Mark Streit,166,48,3.454209,Game of Doans,D,19,1.423439,0.786318,1.077181,-0.792625,1.366975,0.754558,-0.166542,-0.995096,NaN,NaN,NaN
